In [1]:
# import dependencies
import pandas as pd
import numpy as np

In [2]:
# load in files

lab_file = pd.read_csv("Resources_raw/lab_data_raw.csv")
rx_file = pd.read_csv("Resources_raw/reactor_data_raw.csv")
ta_file = pd.read_csv("Resources_raw/towerA_data_raw.csv")
tb_file = pd.read_csv("Resources_raw/towerB_data_raw.csv")
tc_file = pd.read_csv("Resources_raw/towerC_data_raw.csv")

# csv to dataframe

lab_data = pd.DataFrame(lab_file)
rx_data = pd.DataFrame(rx_file)
ta_data = pd.DataFrame(ta_file)
tb_data = pd.DataFrame(tb_file)
tc_data = pd.DataFrame(tc_file)

In [3]:
# display original columns names
#list(lab_data)
list(rx_data)
#list(ta_data)
#list(tb_data)
#list(tc_data)

['Date',
 'RX.TB.RECYCLE.CHARGE.BPD',
 'RX.TB.RECYCLE.CHARGE.PCT',
 'RX.NORTH.TC.CHARGE.BPD',
 'RX.NORTH.TC.CHARGE.CV.PCT',
 'RX.NORTH.FLUSH.PSIG',
 'RX.NORTH.OUT.F',
 'RX.SOUTH.TA.RECYCLE.CHARGE.BPD',
 'RX.SOUTH.TA.RECYCLE.CHARGE.PCT',
 'RX.SOUTH.TC.CHARGE.BPD',
 'RX.SOUTH.TC.CHARGE.CV.PCT',
 'RX.SOUTH.FLUSH.PSIG',
 'RX.SOUTH.OUT.F',
 'RX.SOUTH.WATERRETURN.F',
 'RX.SOUTH.WATERRETURN.PH',
 'RX.OUT.DELTA.F']

In [4]:
# make list of new column names for each dataframe
lab_names = ['Date', 
             'N_Butane']

rx_names = ['Date', 
            'TB_Recycle_Charge', 
            'TB_Recycle_Charge_PCT', 
            'NorthTC_Charge', 
            'NorthTC_Charge_PCT', 
            'North_Flush_Pressure', 
            'North_Out_Temp', 
            'SouthTA_Recycle_Charge', 
            'SouthTA_Recycle_Charge_PCT', 
            'SouthTC_Charge', 
            'SouthTC_Charge_PCT', 
            'South_Flush_Pressure', 
            'South_Out_Temp', 
            'South_Water_Return_Temp', 
            'South_Water_Return_PH', 
            "Out_Delta_Temp"]

ta_names = ['Date', 
            'TA_Charge', 
            'TA_Overhead_Pressure', 
            'TA_Tray4_Temp', 
            'TA_Overhead_Temp', 
            'TA_Recieved_PCT', 
            'TA_Received_Pressure', 
            'TA_Reflux', 
            'TA_Recycle_to_Contact', 
            'TA_Recycle_to_Contact_Temp', 
            'TA_Tray39_Temp', 
            'TA_Tray45_Temp', 
            'TA_Differential_Pressure', 
            'TA_Bottom_Pressure', 
            'TA_Bottom_lvl_PCT', 
            'TA_Bottom_Reboil_Temp', 
            'TA_Top_Reboil_Temp']

tb_names = ['Date',
             'TB_Charge',
             'TB_Charge_Condensor',
             'TB_Charge_Temp',
             'TB_Overhead_Temp',
             'TB_Overhead_Pressure',
             'TB_Overhead_Receiver_Pressure',
             'TB_Overhead_Receiver_Temp',
             'TB_Overhead_Reciever_LVL_PCT',
             'TB_Bottom_Pressure',
             'TB_Bottom_LVL_PCT',
             'TB_Differential_Pressure',
             'TB_Bottom_Temp',
             'TB_Reboil_Out_Temp']

tc_names = ['Date',
             'TC_Tray7_Temp',
             'TC_Tray40_Temp',
             'TC_Charge_Temp',
             'TC_Overhead_Pressure',
             'TC_Overhead_Reciever_Temp',
             'TC_Tray24_Temp',
             'TC_Differential_Pressure',
             'TC_Bottom_Pressure',
             'TC_Reboil_Out_Temp',
             'TC_Reboil_Condensor_lbs']

In [5]:
# make sure that the df lengths match the list lengths
print(f'data: {len(lab_data.columns)}, names: {len(lab_names)}')
print(f'data: {len(rx_data.columns)}, names: {len(rx_names)}')
print(f'data: {len(ta_data.columns)}, names: {len(ta_names)}')
print(f'data: {len(tb_data.columns)}, names: {len(tb_names)}')
print(f'data: {len(tc_data.columns)}, names: {len(tc_names)}')


data: 2, names: 2
data: 16, names: 16
data: 17, names: 17
data: 14, names: 14
data: 11, names: 11


In [6]:
# rename columns
lab_data.columns = lab_names
rx_data.columns = rx_names
ta_data.columns = ta_names
tb_data.columns = tb_names
tc_data.columns = tc_names

In [7]:
# check if renaming worked
print(lab_data.columns)
print(rx_data.columns)

Index(['Date', 'N_Butane'], dtype='object')
Index(['Date', 'TB_Recycle_Charge', 'TB_Recycle_Charge_PCT', 'NorthTC_Charge',
       'NorthTC_Charge_PCT', 'North_Flush_Pressure', 'North_Out_Temp',
       'SouthTA_Recycle_Charge', 'SouthTA_Recycle_Charge_PCT',
       'SouthTC_Charge', 'SouthTC_Charge_PCT', 'South_Flush_Pressure',
       'South_Out_Temp', 'South_Water_Return_Temp', 'South_Water_Return_PH',
       'Out_Delta_Temp'],
      dtype='object')


In [8]:
def clean_data(df):
    
    # make list of df columns
    df_columns = list(df)
    
    # convert necessary objects to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # add hour column for sorting invalid checkpoints
    df['Hour'] = pd.to_datetime(df['Date']).dt.hour
    
    # keep only valid hour checkpoints
    df = df.loc[(df['Hour'] == 6) | (df['Hour'] == 13)]
    
    # store start and end dates of turnaround as variables
    start = "2018-10-12 13:00:00"
    end = "2019-03-29 6:00:00"
    
    # dates between start and end converted to null
    df.loc[(df['Date'] > start) & (df['Date'] <= end)] = np.nan
    
    # for loop to nullify text strings
    for name in df_columns:
        df.loc[df[name] == "I/O Timeout"] = np.nan
        df.loc[df[name] == "Error"] = np.nan
        df.loc[df[name] == "Bad"] = np.nan
        df.loc[df[name] == "Bad Input"] = np.nan
        df.loc[df[name] == "No Data"] = np.nan
        
    # drop nulls
    df = df.dropna()
    
    # convert all other object types to float
    df = df.astype(dict.fromkeys(df.columns[1:-1], float))
    
    # reset index
    df = df.reset_index()
    
    # drop unneeded columns (index and hour)
    df = df.drop(columns=['Hour', 'index'])
    
    return df

In [9]:
# clean lab data
lab_data_clean = clean_data(lab_data)

c:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
lab_data_clean.describe()

,N_Butane
count,5342.000000
mean,11.136153
std,14.474349
min,0.000000
25%,8.200000
50%,10.300000
75%,12.800000
max,1005.000000


In [11]:
# clean reactor data
rx_data_clean = clean_data(rx_data)
rx_data_clean.describe()

c:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TB_Recycle_Charge,TB_Recycle_Charge_PCT,NorthTC_Charge,NorthTC_Charge_PCT,North_Flush_Pressure,North_Out_Temp,SouthTA_Recycle_Charge,SouthTA_Recycle_Charge_PCT,SouthTC_Charge,SouthTC_Charge_PCT,South_Flush_Pressure,South_Out_Temp,South_Water_Return_Temp,South_Water_Return_PH,Out_Delta_Temp
count,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000
mean,2761.123859,28.004040,2685.611161,23.140521,131.203216,91.476504,6768.491359,55.275676,2068.195815,18.320759,131.427272,92.292182,78.365939,7.209025,-0.797493
std,1097.880255,10.762031,550.593059,8.099408,13.792705,7.731235,1020.313107,4.901060,437.958774,5.009712,20.062681,7.517265,7.345467,1.884970,3.335101
min,0.000000,-6.900000,0.000000,-6.900001,0.000000,39.115906,0.000000,-6.900001,0.000000,-6.900001,0.000000,53.540039,51.538086,-0.089193,-47.111736
25%,2046.409607,18.000000,2416.367248,18.177530,124.570370,86.833942,6314.628174,53.140571,1864.641632,16.280122,118.794918,87.878038,74.310684,6.520175,-2.876673
50%,2635.391357,27.163007,2678.655395,21.217908,127.410889,92.583805,6508.248535,55.221201,2104.441407,18.652915,124.353680,93.508320,80.143162,7.547871,-0.276517
75%,3465.538757,35.668804,2992.078308,29.855751,133.004734,97.124060,6843.628052,57.215943,2343.136658,21.153531,141.758537,97.777939,83.795166,8.324176,0.891529
max,7763.305664,105.000000,4402.867188,100.000000,252.685547,110.000893,16473.312500,68.686150,3701.582275,45.000000,252.983093,109.204102,93.135925,14.439209,46.707916


In [12]:
# clean Tower A data
ta_data_clean = clean_data(ta_data)
ta_data_clean.describe()

c:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TA_Charge,TA_Overhead_Pressure,TA_Tray4_Temp,TA_Overhead_Temp,TA_Recieved_PCT,TA_Received_Pressure,TA_Reflux,TA_Recycle_to_Contact,TA_Recycle_to_Contact_Temp,TA_Tray39_Temp,TA_Tray45_Temp,TA_Differential_Pressure,TA_Bottom_Pressure,TA_Bottom_lvl_PCT,TA_Bottom_Reboil_Temp,TA_Top_Reboil_Temp
count,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000
mean,15624.428569,97.724862,134.155713,131.522996,9.290074,100.181961,6992.693161,6726.644595,84.438646,164.226270,178.433037,7.278620,103.291009,90.847109,309.604144,210.518725
std,4264.826486,6.224325,3.587819,4.000799,9.003224,4.538600,1307.059555,989.960174,8.754896,6.900231,10.476421,9.096112,10.362998,12.979410,12.242742,17.863358
min,0.000000,-1.532560,102.691650,102.038574,-6.900000,57.708740,-836.645019,0.000000,27.513123,131.103516,136.929321,0.002177,3.174019,0.000000,185.351562,150.372314
25%,12189.267090,96.931414,132.427978,129.534149,6.732341,99.589539,6351.218872,6297.223755,79.359818,160.897350,174.343872,5.144447,99.865723,82.110928,301.676941,197.711182
50%,15698.013670,97.401123,133.938599,131.687927,9.928573,100.012207,7143.522216,6497.671875,85.122681,164.300552,177.410904,7.573607,104.711151,87.989479,310.391235,207.223518
75%,17979.037597,99.453428,135.274635,133.263462,13.606769,100.360107,7898.555787,6777.425781,89.757053,167.592548,181.070332,8.386230,106.025507,102.352913,319.514465,219.818130
max,29810.134770,133.076477,194.899170,194.660858,105.000000,136.087036,12522.957030,16473.312500,113.681038,299.072266,331.774933,101.661385,141.533661,199.996948,359.384460,329.602081


In [13]:
tb_data_clean = clean_data(tb_data)
tb_data_clean.describe()
# note that lb/hr means pounds per hour

c:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TB_Charge,TB_Charge_Condensor,TB_Charge_Temp,TB_Overhead_Temp,TB_Overhead_Pressure,TB_Overhead_Receiver_Pressure,TB_Overhead_Receiver_Temp,TB_Overhead_Reciever_LVL_PCT,TB_Bottom_Pressure,TB_Bottom_LVL_PCT,TB_Differential_Pressure,TB_Bottom_Temp,TB_Reboil_Out_Temp
count,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000
mean,12772.576421,3647.188501,145.618049,175.237906,183.310000,184.814844,75.782512,61.112795,186.900032,82.212531,3.593235,213.325288,216.348596
std,1064.448754,787.050647,7.450779,5.093775,2.388552,3.063238,19.327296,46.662511,4.035574,4.265610,3.131000,5.145277,4.903160
min,5858.637207,914.653503,123.025520,154.954483,157.860306,155.394135,19.893646,-91.756966,155.940903,63.886410,-2.833572,196.509537,199.824417
25%,12141.566892,3093.009827,138.095253,172.446308,182.816875,183.020649,62.677482,40.789262,184.409168,79.995989,1.729459,209.914875,213.086197
50%,12822.943845,3731.380493,145.350647,175.749786,183.068848,184.525085,77.493286,45.076784,186.549385,84.102146,3.474136,212.543175,216.151367
75%,13393.766843,4221.606689,152.993515,178.372959,183.994247,185.989403,89.450455,67.708408,188.581070,85.054365,4.949290,217.817627,220.166130
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,199.990845,218.297592,110.159851,27.846790,248.548935,257.684631


In [14]:
tc_data_clean = clean_data(tc_data)
tc_data_clean.describe()

c:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TC_Tray7_Temp,TC_Tray40_Temp,TC_Charge_Temp,TC_Overhead_Pressure,TC_Overhead_Reciever_Temp,TC_Tray24_Temp,TC_Differential_Pressure,TC_Bottom_Pressure,TC_Reboil_Out_Temp,TC_Reboil_Condensor_lbs
count,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000
mean,192.221469,204.439332,166.386492,238.913951,171.965331,188.931519,3.278840,242.184423,206.660325,7025.295286
std,4.610535,4.305985,15.046051,9.563152,8.414807,13.632097,0.514274,9.620863,4.051974,1058.005927
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641,50.172615
25%,191.162556,203.845978,158.068848,236.864265,167.680370,173.988453,3.032314,240.154251,205.786133,6343.553223
50%,192.697708,204.798286,164.086914,239.973762,172.128960,195.822716,3.288918,243.133560,206.711647,7005.793213
75%,193.999901,205.913528,170.924381,242.884811,177.360535,197.891464,3.519506,246.192932,207.887211,7751.950928
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584,11856.028320


# Handling Outliers

#### Group Discussion 
- drop rows where RECV.PCT is outside 0-100, indicates invalid
- drop entire Reflux column, an untrusted measure industry wide according to John
- Recycle measures should never be negative, so drop rows where negative

In [15]:
# define find outlier function
def find_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(.75)
    iqr = q3-q1
    lower_lim = q1 - (1.5 * iqr)
    upper_lim = q3 + (1.5 * iqr)
    outliers = df[((df<lower_lim) | (df>upper_lim))]
    return outliers

### Lab Outliers

In [16]:
# outliers in lab data
lab_outliers = find_outliers(lab_data_clean['N_Butane'])
lab_outliers.describe()

count     224.000000
mean       24.769196
std        67.705369
min         0.000000
25%        19.799999
50%        21.799999
75%        26.125000
max      1005.000000
Name: N_Butane, dtype: float64

In [17]:
# set upper limit of 40 for valid values in lab data
lab_data_clean.loc[lab_data_clean['N_Butane'] >= 40] = np.nan
lab_outliers = find_outliers(lab_data_clean['N_Butane'])
lab_outliers.describe()

count    211.000000
mean      18.107109
std       11.065566
min        0.000000
25%        1.100000
50%       21.600000
75%       25.250000
max       36.799999
Name: N_Butane, dtype: float64

### Reactor Outliers

In [18]:
# outliers in reactor data
rx_float = rx_data_clean.iloc[:, 1:-1]
rx_outliers = find_outliers(rx_float)
rx_outliers.describe()

,TB_Recycle_Charge,TB_Recycle_Charge_PCT,NorthTC_Charge,NorthTC_Charge_PCT,North_Flush_Pressure,North_Out_Temp,SouthTA_Recycle_Charge,SouthTA_Recycle_Charge_PCT,SouthTC_Charge,SouthTC_Charge_PCT,South_Flush_Pressure,South_Out_Temp,South_Water_Return_Temp,South_Water_Return_PH
count,15.000000,5.000000,124.000000,60.000000,705.000000,41.000000,749.000000,155.000000,115.000000,136.000000,116.000000,54.000000,90.000000,381.000000
mean,6223.321908,86.409399,1855.541644,-0.172941,151.041288,65.859691,8040.687895,54.443368,813.283946,5.624394,158.409980,69.374049,57.363842,2.863460
std,616.697033,22.077870,1587.513709,20.335159,22.155998,7.258679,1806.014646,21.401906,1249.253469,16.006159,66.713228,4.555457,2.004713,1.806390
min,5624.084473,62.445980,0.000000,-6.900001,0.000000,39.115906,0.000000,-6.900001,0.000000,-6.900001,0.000000,53.540039,51.538086,-0.089193
25%,5792.404297,62.524685,518.433357,-6.900000,148.834228,64.468620,7952.968750,46.237272,5.901617,-6.900000,177.064938,67.960739,55.814365,2.353659
50%,6126.708984,97.076332,1352.204163,-6.900000,151.706696,68.870544,8236.875977,64.469925,10.188961,-3.444444,179.643044,71.173859,57.961275,2.672119
75%,6397.888184,105.000000,3933.201782,-6.500000,156.762741,70.547485,8702.639648,66.946980,1027.408783,8.833701,184.145519,72.438751,59.090427,2.993232
max,7763.305664,105.000000,4402.867188,100.000000,252.685547,71.394348,16473.312500,68.686150,3701.582275,45.000000,252.983093,73.013611,60.006714,14.439209


In [19]:
# null invalid values
rx_data_clean.loc[(rx_data_clean['NorthTC_Charge_PCT'] < 0) | (rx_data_clean['NorthTC_Charge_PCT'] > 100)] = np.nan
rx_data_clean.loc[(rx_data_clean['TB_Recycle_Charge_PCT'] < 0) | (rx_data_clean['TB_Recycle_Charge_PCT'] > 100)] = np.nan
rx_data_clean.loc[(rx_data_clean['SouthTA_Recycle_Charge_PCT'] < 0) | (rx_data_clean['SouthTA_Recycle_Charge_PCT'] > 100)] = np.nan
rx_data_clean.loc[(rx_data_clean['SouthTC_Charge_PCT'] < 0) | (rx_data_clean['SouthTC_Charge_PCT'] > 100)] = np.nan
rx_data_clean.loc[rx_data_clean['TB_Recycle_Charge'] == 0] = np.nan
rx_data_clean.loc[rx_data_clean['NorthTC_Charge'] == 0] = np.nan
rx_data_clean.loc[rx_data_clean['SouthTC_Charge'] == 0] = np.nan

In [20]:
# drop nulls and unnecessary columns and check dataframe
rx_data_clean = rx_data_clean.dropna()

# drop unnecessary columns for machine learning
# Reasoning: The Percent columns were useful for identifying entire rows that
# were likely to be invalid, however, the percent values themselves are not
# needed since we have the numerical values as features too
rx_data_clean = rx_data_clean.drop(columns=['TB_Recycle_Charge_PCT',
                                           'NorthTC_Charge_PCT',
                                           'SouthTA_Recycle_Charge_PCT',
                                           'SouthTC_Charge_PCT'])

rx_data_clean.describe()

,TB_Recycle_Charge,NorthTC_Charge,North_Flush_Pressure,North_Out_Temp,SouthTA_Recycle_Charge,SouthTC_Charge,South_Flush_Pressure,South_Out_Temp,South_Water_Return_Temp,South_Water_Return_PH,Out_Delta_Temp
count,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000,3963.000000
mean,2759.437045,2714.933315,131.524328,91.674697,6798.186154,2102.703469,131.760762,92.621904,78.474800,7.183606,-0.927418
std,1068.141433,492.332154,11.602840,7.399534,872.424494,337.779018,17.842611,7.182914,7.335540,1.895262,2.289875
min,6.672578,784.665832,107.486725,62.329102,1505.005737,68.151711,34.842491,57.240231,51.538086,-0.089193,-20.792578
25%,2049.087524,2428.814209,124.626156,87.081791,6319.610596,1880.577759,118.884090,88.445282,74.493587,6.512169,-2.879174
50%,2632.141357,2686.425049,127.395638,92.678841,6508.779297,2112.691650,124.343842,93.740868,80.314636,7.527303,-0.330758
75%,3448.883057,3002.929931,132.839203,97.149864,6845.254639,2346.403076,141.506195,97.881741,83.853149,8.319141,0.877244
max,7322.576172,4402.867188,242.542267,110.000893,16019.025390,3565.502930,203.495590,109.204102,93.135925,14.439209,46.707916


In [21]:
# recheck outliers
rx_float = rx_data_clean.iloc[:, 1:-1]
rx_outliers = find_outliers(rx_float)
rx_outliers.describe()

,TB_Recycle_Charge,NorthTC_Charge,North_Flush_Pressure,North_Out_Temp,SouthTA_Recycle_Charge,SouthTC_Charge,South_Flush_Pressure,South_Out_Temp,South_Water_Return_Temp,South_Water_Return_PH
count,10.000000,79.000000,681.000000,33.000000,699.000000,33.000000,102.000000,49.000000,94.000000,379.000000
mean,5982.541455,2732.219845,153.481965,69.533068,8281.025284,1580.231815,179.873052,71.336644,57.639298,2.830434
std,529.373379,1336.582481,8.880683,2.505413,1095.804955,1061.582843,15.512981,3.507231,2.027629,1.709530
min,5564.300781,784.665832,107.486725,62.329102,1505.005737,68.151711,34.842491,57.240231,51.538086,-0.089193
25%,5664.665039,1448.358520,148.709701,68.644714,7986.348389,973.041931,177.615807,71.159363,56.107330,2.351668
50%,5792.404297,3879.869629,151.615021,70.396423,8254.931641,1116.073608,179.705994,72.654724,58.210758,2.670792
75%,6109.588623,3987.058594,156.544815,71.240234,8760.738281,3067.762695,183.088512,73.365784,59.273149,2.981954
max,7322.576172,4402.867188,242.542267,71.766663,16019.025390,3565.502930,203.495590,74.223328,60.409546,14.439209


### Tower A Outliers

In [22]:
# Tower A outliers
ta_float = ta_data_clean.iloc[:, 1:-1]
ta_outliers = find_outliers(ta_float)
ta_outliers.describe()

,TA_Charge,TA_Overhead_Pressure,TA_Tray4_Temp,TA_Overhead_Temp,TA_Recieved_PCT,TA_Received_Pressure,TA_Reflux,TA_Recycle_to_Contact,TA_Recycle_to_Contact_Temp,TA_Tray39_Temp,TA_Tray45_Temp,TA_Differential_Pressure,TA_Bottom_Pressure,TA_Bottom_lvl_PCT,TA_Bottom_Reboil_Temp
count,41.000000,949.000000,334.000000,286.000000,665.000000,1538.000000,172.000000,814.000000,110.000000,96.000000,178.000000,631.000000,255.000000,57.000000,33.000000
mean,17700.180372,96.437474,139.539384,133.438247,-0.987673,100.473771,4308.519581,7986.333103,75.129156,167.194391,199.327128,7.829190,100.460613,72.849561,291.339584
std,13488.224614,12.995649,8.944124,11.769573,16.949805,7.677170,3383.735933,1752.429110,22.847436,33.895166,40.552331,23.522877,40.439689,61.702114,47.614630
min,0.000000,-1.532560,102.691650,102.038574,-6.900000,57.708740,-836.645019,0.000000,27.513123,131.103516,136.929321,0.002177,3.174019,0.000000,185.351562
25%,43.794273,92.136383,139.643097,122.940832,-6.900000,95.019531,2154.883484,7818.320435,60.141754,147.636414,163.650513,0.104206,115.336994,21.635437,270.324707
50%,26958.464840,92.500305,141.523743,139.385986,-6.900000,97.713470,3206.244751,8201.000977,62.731173,149.954224,192.954834,0.180817,117.974853,39.322662,272.357178
75%,27801.376950,105.022430,143.545532,141.210937,-6.900000,104.817200,3903.295838,8594.729981,105.738394,180.420689,201.997387,0.245193,122.032932,142.654419,348.138428
max,29810.134770,133.076477,194.899170,194.660858,105.000000,136.087036,12522.957030,16473.312500,113.681038,299.072266,331.774933,101.661385,141.533661,199.996948,359.384460


In [23]:
# null out dataset specific invalid values 
ta_data_clean.loc[(ta_data_clean['TA_Recieved_PCT'] < 0) | (ta_data_clean['TA_Recieved_PCT'] > 100)] = np.nan
ta_data_clean.loc[ta_data_clean['TA_Charge'] < 5000] = np.nan
ta_data_clean.loc[ta_data_clean['TA_Recycle_to_Contact'] < 1000] = np.nan
ta_data_clean.loc[(ta_data_clean['TA_Bottom_lvl_PCT'] < 0) | (ta_data_clean['TA_Bottom_lvl_PCT'] > 100)] = np.nan

In [24]:
# drop nulls
ta_data_clean = ta_data_clean.dropna()

# drop unnecessary columns for machine learning
ta_data_clean = ta_data_clean.drop(columns=['TA_Reflux'])

ta_data_clean.describe()

,TA_Charge,TA_Overhead_Pressure,TA_Tray4_Temp,TA_Overhead_Temp,TA_Recieved_PCT,TA_Received_Pressure,TA_Recycle_to_Contact,TA_Recycle_to_Contact_Temp,TA_Tray39_Temp,TA_Tray45_Temp,TA_Differential_Pressure,TA_Bottom_Pressure,TA_Bottom_lvl_PCT,TA_Bottom_Reboil_Temp,TA_Top_Reboil_Temp
count,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000
mean,13581.226564,97.587107,133.368885,130.993651,12.191080,99.894853,6437.945549,83.763715,163.530380,178.899115,6.968114,101.580655,82.201310,316.089147,206.034508
std,3118.119523,5.398608,2.173454,2.990626,6.556598,0.977370,525.975330,7.362435,5.297077,6.641016,11.786154,11.974523,8.621321,8.202325,13.753964
min,5245.315430,-1.532560,124.008186,116.775520,0.057902,88.854980,2651.605713,54.576115,144.874649,161.386108,0.002177,3.174019,0.000000,278.997803,174.914551
25%,11102.405520,97.143854,132.234192,129.747009,8.958455,99.813562,6274.797119,79.589844,160.389709,175.003052,0.375391,99.817751,79.637146,310.722725,196.352386
50%,12901.359375,97.443008,133.589172,131.848144,11.660881,100.009163,6451.078125,84.451801,163.805000,178.778534,7.617188,103.851700,84.366089,317.794800,203.393570
75%,15862.798585,99.104103,134.766972,132.995617,15.454600,100.195312,6520.216431,88.112961,167.251793,182.182857,8.537292,105.797768,85.597305,321.911621,213.596851
max,29810.134770,109.439850,145.645142,140.570068,83.057693,112.509163,16473.312500,108.530273,177.632065,255.816666,101.661385,127.102669,99.871826,356.948029,271.478272


In [25]:
# recheck outliers after dropping invalids
ta_float = ta_data_clean.iloc[:, 1:-1]
ta_outliers = find_outliers(ta_float)
ta_outliers.describe()

,TA_Charge,TA_Overhead_Pressure,TA_Tray4_Temp,TA_Overhead_Temp,TA_Recieved_PCT,TA_Received_Pressure,TA_Recycle_to_Contact,TA_Recycle_to_Contact_Temp,TA_Tray39_Temp,TA_Tray45_Temp,TA_Differential_Pressure,TA_Bottom_Pressure,TA_Bottom_lvl_PCT,TA_Bottom_Reboil_Temp
count,19.000000,54.000000,86.000000,172.000000,50.000000,323.000000,157.000000,85.000000,29.000000,83.000000,52.000000,64.000000,150.000000,30.000000
mean,25611.716078,80.752185,131.368875,123.680621,34.990243,98.991712,6902.864448,84.550297,148.964808,185.795882,81.372678,44.031318,69.274116,294.764314
std,2224.087842,32.143951,6.071902,3.051906,15.063984,2.468638,1960.231966,20.392563,5.675067,19.928741,22.118230,44.348801,26.821873,15.368648
min,23086.044920,-1.532560,124.008186,116.775520,25.298895,88.854980,2651.605713,54.576115,144.874649,161.386108,20.792770,3.174019,0.000000,278.997803
25%,23969.040040,91.533857,127.500155,122.608948,26.361126,97.935734,5651.323730,64.535522,147.250366,163.491821,74.184895,3.243801,62.910081,290.180168
50%,24803.482420,92.546082,128.153992,123.696899,28.876105,98.980637,5904.003418,101.150459,148.110718,193.983215,94.001663,23.390293,69.876862,292.404175
75%,27095.091800,93.501669,138.823917,124.354553,35.196927,100.986183,7317.470703,103.171722,149.037933,198.274277,94.335812,87.094116,95.266724,293.598938
max,29810.134770,109.439850,145.645142,140.570068,83.057693,112.509163,16473.312500,108.530273,177.632065,255.816666,101.661385,127.102669,99.871826,356.948029


### Tower B Outliers

In [26]:
# Tower B outliers
tb_float = tb_data_clean.iloc[:, 1:-1]
tb_outliers = find_outliers(tb_float)
tb_outliers.describe()

,TB_Charge,TB_Charge_Condensor,TB_Charge_Temp,TB_Overhead_Temp,TB_Overhead_Pressure,TB_Overhead_Receiver_Pressure,TB_Overhead_Receiver_Temp,TB_Overhead_Reciever_LVL_PCT,TB_Bottom_Pressure,TB_Bottom_LVL_PCT,TB_Differential_Pressure,TB_Bottom_Temp
count,42.000000,4.000000,1.000000,49.000000,336.000000,107.000000,16.000000,215.000000,65.000000,22.000000,45.000000,12.000000
mean,11480.498152,4977.103561,185.386932,161.901258,183.292304,190.137803,126.420147,141.342127,197.276243,70.389702,15.877803,205.069527
std,3348.236932,2739.297348,NaN,9.354830,5.048110,7.766866,28.486220,82.272784,10.757208,9.423146,5.301888,19.419842
min,5858.637207,914.653503,185.386932,154.954483,157.860306,155.394135,19.893646,-91.756966,155.940903,63.886410,10.317993,196.509537
25%,9221.930176,4681.486618,185.386932,157.891846,179.644089,190.818596,131.841183,123.170505,195.703110,64.787657,11.676937,196.602062
50%,10084.878905,6047.801025,185.386932,160.096832,180.315941,192.013550,133.478470,183.132752,198.195862,70.378967,13.531866,196.756477
75%,15484.844730,6343.417969,185.386932,162.020874,187.259766,194.374870,134.030819,199.990845,202.056900,71.240433,19.363403,197.046520
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,199.990845,218.297592,110.159851,27.846790,248.548935


In [27]:
# remove invalid rows from dataset (anything PCT needs to be between 0-100)
tb_data_clean.loc[(tb_data_clean['TB_Overhead_Reciever_LVL_PCT'] < 0) | (tb_data_clean['TB_Overhead_Reciever_LVL_PCT'] > 100)] = np.nan
tb_data_clean.loc[(tb_data_clean['TB_Bottom_LVL_PCT'] < 0) | (tb_data_clean['TB_Bottom_LVL_PCT'] > 100)] = np.nan

In [28]:
# drop nulls and check dataframe again
tb_data_clean = tb_data_clean.dropna()

# drop unnecessary columns for machine learning
# no unnecessary columns

tb_data_clean.describe()

,TB_Charge,TB_Charge_Condensor,TB_Charge_Temp,TB_Overhead_Temp,TB_Overhead_Pressure,TB_Overhead_Receiver_Pressure,TB_Overhead_Receiver_Temp,TB_Overhead_Reciever_LVL_PCT,TB_Bottom_Pressure,TB_Bottom_LVL_PCT,TB_Differential_Pressure,TB_Bottom_Temp,TB_Reboil_Out_Temp
count,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000
mean,12888.041948,3606.667269,145.199813,174.998975,183.264743,184.963378,77.269241,47.744934,186.898937,82.056868,3.648800,213.244295,216.401481
std,1047.337377,773.070854,7.393804,4.726962,2.265965,2.808644,18.705931,18.759088,3.719031,4.435992,2.857346,5.187898,4.899155
min,5858.637207,914.653503,123.025520,154.954483,173.002945,170.526138,19.893646,0.000000,173.029419,63.886410,-2.833572,196.509537,199.824417
25%,12304.933353,3070.760315,138.064068,172.332779,182.793419,183.076859,65.463675,40.154741,184.677052,79.788177,2.040822,209.905628,213.185089
50%,12931.493655,3686.642212,144.931732,175.427246,183.068848,184.658653,78.659184,44.763899,186.545570,84.248760,3.517921,212.338257,216.130386
75%,13492.972655,4158.917114,152.969051,177.870483,183.984573,186.053436,89.846333,56.918225,188.554207,85.078440,4.985080,217.725941,220.143753
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,99.378967,218.297592,91.452347,27.846790,248.548935,257.684631


In [29]:
# recheck outliers
tb_float = tb_data_clean.iloc[:, 1:-1]
tb_outliers = find_outliers(tb_float)
tb_outliers.describe()

,TB_Charge,TB_Charge_Condensor,TB_Charge_Temp,TB_Overhead_Temp,TB_Overhead_Pressure,TB_Overhead_Receiver_Pressure,TB_Overhead_Receiver_Temp,TB_Overhead_Reciever_LVL_PCT,TB_Bottom_Pressure,TB_Bottom_LVL_PCT,TB_Differential_Pressure,TB_Bottom_Temp
count,39.000000,4.000000,1.000000,40.000000,294.000000,80.000000,22.000000,153.000000,53.000000,18.000000,33.000000,12.000000
mean,11675.613169,4977.103561,185.386932,163.953601,183.080209,192.041041,108.048514,52.923849,197.469664,67.920769,13.479817,205.069527
std,3402.555396,2739.297348,NaN,10.545475,4.750508,5.511911,46.808270,41.191007,9.375584,3.125278,8.848158,19.419842
min,5858.637207,914.653503,185.386932,154.954483,173.002945,170.526138,19.893646,0.000000,173.029419,63.886410,-2.833572,196.509537
25%,9266.100097,4681.486618,185.386932,159.595379,179.610222,191.193809,127.306509,6.935675,194.945999,64.745235,10.469055,196.602062
50%,10180.187500,6047.801025,185.386932,161.683586,180.163246,192.365654,132.110596,82.911995,197.442642,69.563980,12.091387,196.756477
75%,15497.792970,6343.417969,185.386932,163.502655,187.070847,194.493809,133.741280,88.106064,201.594116,70.649614,19.308472,197.046520
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,99.378967,218.297592,71.662560,27.846790,248.548935


### Tower C Outliers

In [30]:
# Tower C outliers
tc_float = tc_data_clean.iloc[:, 1:-1]
tc_outliers = find_outliers(tc_float)
tc_outliers.describe()

# NOTE: Values here all fall within a relativley normal and expected range
# no drops necessary

,TC_Tray7_Temp,TC_Tray40_Temp,TC_Charge_Temp,TC_Overhead_Pressure,TC_Overhead_Reciever_Temp,TC_Tray24_Temp,TC_Differential_Pressure,TC_Bottom_Pressure,TC_Reboil_Out_Temp
count,152.000000,266.000000,551.000000,266.000000,115.000000,14.000000,156.000000,261.000000,244.000000
mean,178.240995,195.843639,187.103888,218.227422,141.746433,141.016376,3.624299,221.062942,203.614664
std,19.985205,15.850930,28.401021,32.011793,17.351797,50.253124,2.180474,32.543254,17.394623
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641
25%,180.039295,198.388329,192.583275,221.793205,136.433159,122.065285,2.170433,224.972626,201.150467
50%,185.117188,199.827560,195.742798,224.469307,145.364380,132.572472,2.284555,227.857727,201.975967
75%,186.369782,200.394325,197.219856,226.440285,150.180054,134.632637,4.545918,229.560928,202.542221
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584


In [31]:
tc_data_clean.describe()

,TC_Tray7_Temp,TC_Tray40_Temp,TC_Charge_Temp,TC_Overhead_Pressure,TC_Overhead_Reciever_Temp,TC_Tray24_Temp,TC_Differential_Pressure,TC_Bottom_Pressure,TC_Reboil_Out_Temp,TC_Reboil_Condensor_lbs
count,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000
mean,192.221469,204.439332,166.386492,238.913951,171.965331,188.931519,3.278840,242.184423,206.660325,7025.295286
std,4.610535,4.305985,15.046051,9.563152,8.414807,13.632097,0.514274,9.620863,4.051974,1058.005927
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641,50.172615
25%,191.162556,203.845978,158.068848,236.864265,167.680370,173.988453,3.032314,240.154251,205.786133,6343.553223
50%,192.697708,204.798286,164.086914,239.973762,172.128960,195.822716,3.288918,243.133560,206.711647,7005.793213
75%,193.999901,205.913528,170.924381,242.884811,177.360535,197.891464,3.519506,246.192932,207.887211,7751.950928
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584,11856.028320


# DF to CSV Pass 1

In [32]:
#lab_data.to_csv("Resources_clean/lab_pass1.csv")
#rx_data.to_csv("Resources_clean/rx_pass1.csv")
#ta_data.to_csv("Resources_clean/ta_pass1.csv")
#tb_data.to_csv("Resources_clean/tb_pass1.csv")
#tc_data.to_csv("Resources_clean/tc_pass1.csv")

# DF to CSV Updated

In [33]:
# check for any remnant nulls I may have missed
#print(lab_data_clean.isnull().sum())
#print(rx_data_clean.isnull().sum())
#print(ta_data_clean.isnull().sum())
#print(tb_data_clean.isnull().sum())
#print(tc_data_clean.isnull().sum())

# drop remnant nulls
lab_data_clean = lab_data_clean.dropna()
rx_data_clean = rx_data_clean.dropna()
ta_data_clean = ta_data_clean.dropna()
tb_data_clean = tb_data_clean.dropna()
tc_data_clean = tc_data_clean.dropna()


In [36]:
print(lab_data_clean.shape)
print(rx_data_clean.shape)
print(ta_data_clean.shape)
print(tb_data_clean.shape)
print(tc_data_clean.shape)

(5329, 2)
(3963, 12)
(2546, 16)
(1416, 14)
(5260, 11)


In [37]:
# drop Date duplicates from all dataframes
lab_data_clean = lab_data_clean.drop_duplicates(subset=['Date'])
rx_data_clean = rx_data_clean.drop_duplicates(subset=['Date'])
ta_data_clean = ta_data_clean.drop_duplicates(subset=['Date'])
tb_data_clean = tb_data_clean.drop_duplicates(subset=['Date'])
tc_data_clean = tc_data_clean.drop_duplicates(subset=['Date'])

In [38]:
print(lab_data_clean.shape)
print(rx_data_clean.shape)
print(ta_data_clean.shape)
print(tb_data_clean.shape)
print(tc_data_clean.shape)

(5327, 2)
(3961, 12)
(2544, 16)
(1416, 14)
(5258, 11)


In [39]:
lab_data_clean.to_csv("Resources_clean/lab_updated.csv")
rx_data_clean.to_csv("Resources_clean/rx_updated.csv")
ta_data_clean.to_csv("Resources_clean/ta_updated.csv")
tb_data_clean.to_csv("Resources_clean/tb_updated.csv")
tc_data_clean.to_csv("Resources_clean/tc_updated.csv")